# Skapa vmodel och model på senaste datat utan test
vmodel anväder volume som input till ret_y

In [1]:
import pandas as pd
from datetime import datetime as dt
import yfinance as yf
import matplotlib.pyplot as plt 
from catboost import CatBoostRegressor,Pool,utils

plt.style.use('fivethirtyeight')
pd.set_option('display.width',200)

In [2]:
def set_kolumner():
    rkolumner = ['vol_y', 'ret_2', 'vol_2', 'ret_3', 'vol_3', 'ret_4', 'vol_4', 'ret_5', 'vol_5', 
                'ret_6', 'vol_6', 'ret_7', 'vol_7', 'ret_8', 'vol_8', 'ret_9', 'vol_9', 'ret_10', 
                'vol_10', 'year', 'month', 'wday', 'day']
    vkolumner = rkolumner[1:]
    
    return vkolumner, rkolumner

# let the data include vol_y as input to ret_y-training

In [3]:
def fit_data(train, iterations=550):
    """ prepare data for training and fit models
    returns: vmodel(uses volume) and model
    """
    vkolumner, rkolumner = set_kolumner()
    
    ret_y=train.ret_y
    vol_y=train.vol_y

    train.to_csv('krypto_df.csv')
    # train[vkolumner].to_csv('train_v.csv')
    
    print('shape train av ret_y',train[rkolumner].shape)   
    print('shape train av vol_y',train[vkolumner].shape)   
    
    vmodel = CatBoostRegressor(iterations=iterations,
        verbose=False, random_state=2021)
    
    vmodel.fit(train[vkolumner],y=vol_y,use_best_model=False) # train on 'kolumner' except vol_y and ret_y
    
    model = CatBoostRegressor(iterations=iterations,
        verbose=False, random_state=2021)
    
    model.fit(train[rkolumner],y=ret_y,use_best_model=False) # includes vol_y for training
    
    return vmodel, model

In [4]:
def full_model( data):
    # data = data.tail(725)  # använd de 725 sista dagarna (2 år)
    vmodel, model = fit_data(data)
    # print(data.shape)
    return vmodel,model

def pct_change(df, ny, gammal, dagar):
    df[ny+'_'+str(dagar)] = df[gammal].pct_change(dagar)

# Historik för proc utveckling av 'Adj Close' och volume
def add_history(df):
    for i in range (10):
        pct_change(df,'ret','Adj Close',i+1)
        pct_change(df,'vol','Volume',i+1)
    
    df.rename(columns={'vol_1':'vol_y', 'ret_1':'ret_y'},inplace=True)
    df = df.dropna()
    return df

# kolumner för år, månad, dag samt predict
def add_more_cols(df):
    df['pred_Volume'] = None
    df['pred_AdjClose'] = None
        
    df['year']=pd.DatetimeIndex(df.index).year
    df['month']=pd.DatetimeIndex(df.index).month
    df['wday']=pd.DatetimeIndex(df.index).weekday
    df['day']=pd.DatetimeIndex(df.index).day
   
    return df


## skapa alla modellerna

In [5]:
""" en modell per valuta """

tickers = ['ETH-USD','BTC-USD','BCH-USD','ZRX-USD','XRP-USD',]

for ticker in tickers:
    print()
    print('ladda ner data för',ticker)
    df = yf.download([ticker], progress=True)[['Adj Close','Volume']]
    # df = df.iloc[-725:]
    df=add_history(df.copy())
    df=add_more_cols(df.copy())
    print(ticker,df.shape)
    
    # skapa direkt 5 dagar framåt
    vmodel, model = full_model(df) # skapa modeller
    vmodel.save_model(f'{ticker[:3]}_vmodel')
    model.save_model(f'{ticker[:3]}_model')
    
    # Alternativa modeller, varje dag-resutat blir input till nästa dag
    df2=df.copy()
    for day in range(5):
        df2[['ret_y','vol_y']] = df2[['ret_y','vol_y']].shift(-1) 
        df2=df2.iloc[:-1,:]
        print(ticker,df2.shape)
        vmodel, model = full_model(df2)
        vmodel.save_model(f'{ticker[:3]}{day+1}_vmodel')
        model.save_model(f'{ticker[:3]}{day+1}_model')
    


ladda ner data för ETH-USD
[*********************100%***********************]  1 of 1 completed
ETH-USD (1763, 28)
shape train av ret_y (1763, 23)
shape train av vol_y (1763, 22)
ETH-USD (1762, 28)
shape train av ret_y (1762, 23)
shape train av vol_y (1762, 22)
ETH-USD (1761, 28)
shape train av ret_y (1761, 23)
shape train av vol_y (1761, 22)
ETH-USD (1760, 28)
shape train av ret_y (1760, 23)
shape train av vol_y (1760, 22)
ETH-USD (1759, 28)
shape train av ret_y (1759, 23)
shape train av vol_y (1759, 22)
ETH-USD (1758, 28)
shape train av ret_y (1758, 23)
shape train av vol_y (1758, 22)

ladda ner data för BTC-USD
[*********************100%***********************]  1 of 1 completed
BTC-USD (2912, 28)
shape train av ret_y (2912, 23)
shape train av vol_y (2912, 22)
BTC-USD (2911, 28)
shape train av ret_y (2911, 23)
shape train av vol_y (2911, 22)
BTC-USD (2910, 28)
shape train av ret_y (2910, 23)
shape train av vol_y (2910, 22)
BTC-USD (2909, 28)
shape train av ret_y (2909, 23)
shape tr